# Stealth address
Ce notebook explicite le notebook [original](https://github.com/nerolation/EIP-Stealth-Address-ERC).

Ici on s'intéresse au cas ou le receiver Bob veut une clé de scan pour l'audit, montrer qu'il est honnête, et une clé de dépense qu'il utilise pour utiliser les tokens qu'il a reçus (d'Alice).

### Rappel sur la cryptographie sur courbe elliptique 
Une courbe elliptique est définie par une équation de type: 


$y^2= ax^3 + bx + c$

avec a,b fixes, x et y les variables du corps fini.
Il s'agit donc d'une fonction à deux variables: chaque point est défini par les coordonnées de x et y : (x,y).

Pour définit un système cryptographique basé sur les courbes elliptiques, on choisit une courbe elliptique en fixant a et b, on choisit un point de la courbe G, qu'on nomme le générateur et qui est publique, et on choisit un maximum du corps fini (pour les modulos). 

On peut additionner deux points facilement mais la multiplication est particulière car multiplier G par 5 par exemple revient à additionner G 5 fois.

Dans les systèmes cryptographiques, on utilise des fonction trapdoors, c'est à dire qu'il est facile de réaliser une opération dans un sens mais quasiment impossible dans l'autre sens. 
En l'occurence, il est facile de multiplier G par 5 pour obtenir un point P, mais à partir de P il n'est pas possible de récupérer le 5 initial. C'est le problème du logarithme discret. 

Cela nous permet de définir une clé privée s, et de communiquer la clé publique S=G*s . A partir de S, il n'est pas possible de récupérer s. 

In [1]:
# PoC using scanning and spending keys

In [2]:
!python -m pip install py_ecc pysha3 eth_account

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import hashlib
from py_ecc.secp256k1 import *
import sha3
from eth_account import Account

## Sender


Ici le sender est Alice, elle veut envoyer des tokens à Bob. Pour cela, elle commence par générer sa clé éphémère $a_{eph}$ et en déduit sa clé publique éphèmère $A_{eph}$. Cette clé ne servira qu'une fois, pour ce transfert. Elle garde secrète sa clé privée éphémère et publie, envoie au registre global des clés éphémères, la clé publique éphémère $A_{eph}$.

$A_{eph} = G*a_{eph}$

In [4]:
# privkey: 0xd952fe0740d9d14011fc8ead3ab7de3c739d3aa93ce9254c10b0134d80d26a30
# address: 0x3CB39EA2f14B16B69B451719A7BEd55e0aFEcE8F
a_eph = int(0xd952fe0740d9d14011fc8ead3ab7de3c739d3aa93ce9254c10b0134d80d26a30) # private key
A_eph = secp256k1.privtopub(a_eph.to_bytes(32, "big")) # public key
A_eph

(22246744184454969143801186698733154500632648736073949898323976612504587645286,
 110772761940586493986212935445517909380300793379795289150161960681985511655321)

## Recipient

Ici le receveur, Bob, génère deux clés privées, une pour lire ses adresses stealth qu'il pourra communiquer en cas d'audit par exemple, une pour les dépenser. On a donc 4 clés : 

*   La clé privée de lecture b_scan
*   La clé publique de lecture B_scan
*   La clé privée de dépense b_spend
*   La clé publique de dépense B_spend

Dans le blog de Vitalik, ces clés publiques correspondent aux stealth meta-address, c'est à dire que ce sont les clés de Bob qui vont lui permettre de lire et dépenser les fonds dans tous les adresses stealth qui lui sont destinées.

Remarque: Le commentaire en première ligne indique juste qu'il est possible de faire le même procédé avec une seule clé de dépense, l'idée initialement expliquée dans le [blog](https://vitalik.ca/general/2023/01/20/stealth.html) de Vitalik. [texte du lien](https://)

$B_{scan} = G*b_{scan}$

$B_{spend} = G*b_{spend}$

In [5]:
# privkey: 0x0000000000000000000000000000000000000000000000000000000000000001
# address: 0x7E5F4552091A69125d5DfCb7b8C2659029395Bdf
b_scan = int(0x0000000000000000000000000000000000000000000000000000000000000002) # private key
b_spend = int(0x0000000000000000000000000000000000000000000000000000000000000003) # private key

B_scan = secp256k1.privtopub(b_scan.to_bytes(32, "big")) # public key
B_spend = secp256k1.privtopub(b_spend.to_bytes(32, "big")) # public key
B_scan, B_spend

((89565891926547004231252920425935692360644145829622209833684329913297188986597,
  12158399299693830322967808612713398636155367887041628176798871954788371653930),
 (112711660439710606056748659173929673102114977341539408544630613555209775888121,
  25583027980570883691656905877401976406448868254816295069919888960541586679410))

## Principe 
Maintenant que Alice a la clé éphémère et Bob a ses clé(s) privée(s), le but est de construire une adresse où Alice peut envoyer des tokens que seul Bob peut récupérer, sans dévoiler évidemment son adresse principale. 
Pour cela, il faut: 
1.   Se mettre d'accord sur une addresse, clé publique, éphémère sur laquelle Alice et Bob vont communiquer. (à la Diffie Hellman)
2.   faire en sorte que seul Bob connaisse la clé privée de cette clé publique éphémère. 

Pour rappel, Diffie Hellman permet à Alice et Bob se mettre d'accord sur une clé que seuls eux-même connaissent, de manière sécurisée. Cela peut servir ensuite pour passer en cryptographie symétrique par exemple. La méthode des adresses stealth en est inspirée mais la modifie pour que seul Bob connaisse la clé privée de l'adresse convenue.  



## Le sender Alice calcule l'adresse stealth de Bob 

Si P_scan est la clé publique de Bob et s la clé privée éphémère d'Alice, Alice peut déterminer l'adresse partagée comme ceci:

$Shared_{key}=B_{scan} * a_{eph}$. 

Elle combine sa clé privée avec la clé publique de scan de Bob.

Cette adresse est partagée car Bob peut également retomber sur le même resultat d'une autre manière:

$Shared_{key}=b_{scan} * A_{eph}$. 



Pour cela, elle calcule d'abord le secret partagé: 

$Shared_{secret} = R * p_{scan}$

In [6]:
Shared = secp256k1.multiply(B_scan, a_eph)
Shared

(65311808848028536848162101908966111079795231803322390815513763038079235257196,
 43767810034999830518515787564234053904327508763526333662117780420755425490082)

Il combine sa clé privé de scan avec la clé publique éphémère d'Alice qu'il a récupéré du registre. 
Cela s'explique par l'associativité de la multiplication:

$Shared_{key} \\ 
= G * b_{scan} * a_{eph} = (G*b_{scan})*a_{eph}= B_{scan} * a_{eph} \\
= G * b_{scan} * a_{eph} =b_{scan}*(G*a_{eph}) = b_{scan} * A_{eph}$

Alice et Bob se sont donc mis d'accord sur une clé/adresse commune, ce qui correspond à du Diffie Hellman.



In [7]:
assert Shared == secp256k1.multiply(A_eph, b_scan)

Alice hash ensuite cette clé partagée

In [8]:
Shared_hex = sha3.keccak_256(Shared[0].to_bytes(32, "big") 
                        + Shared[1].to_bytes(32, "big")
                       ).hexdigest()
Shared_hased = bytearray.fromhex(Shared_hex)


Maintenant, il faut modifier cette adresse pour que seul Bob connaisse la clé privée. Pour cela Alice la hash et la combine avec la clé publique de dépense de Bob. 

$ Stealth_{key}=B_{spend} + G * hash(Shared_{key})$


In [9]:
Stealth_key = secp256k1.add(B_spend, secp256k1.privtopub(Shared_hased))
A_stealth_address = "0x"+ sha3.keccak_256(Stealth_key[0].to_bytes(32, "big")
                            +Stealth_key[1].to_bytes(32, "big")
                           ).hexdigest()[-40:]
A_stealth_address

'0xfed69df0a27f1dae0d7430ead82aaedfad6332bb'

#### Alice envoie les tokens à cette adresse stealth  

#### Ensuite elle envoie la clé publique éphémère et l'adresse stealth au registre de clés éphémères.

In [10]:
A_eph, A_stealth_address

((22246744184454969143801186698733154500632648736073949898323976612504587645286,
  110772761940586493986212935445517909380300793379795289150161960681985511655321),
 '0xfed69df0a27f1dae0d7430ead82aaedfad6332bb')

## Bob veut maintenant récupérer les fonds

Pour cela il calcule d'abord la clé partagée à l'aide de sa clé privée de dépense:

$Shared_{key}= G * b_{scan} * a_{eph} =b_{scan}*(G*a_{eph}) = b_{scan} * A_{eph}$

Comme Alice il hashe cette clé partagée et ajoute sa clé publique de dépense:

$ Stealth_{key}=B_{spend} + G * hash(Shared_{key})$


In [11]:
Shared = secp256k1.multiply(A_eph, b_scan)
Shared_hex = sha3.keccak_256(Shared[0].to_bytes(32, "big")+Shared[1].to_bytes(32, "big")).hexdigest()
Shared_hased = bytearray.fromhex(Shared_hex)

B_stealth = secp256k1.add(B_spend, secp256k1.privtopub(Shared_hased))
B_stealth_address  = "0x"+ sha3.keccak_256(Stealth_key[0].to_bytes(32, "big")
                                        + Stealth_key[1].to_bytes(32, "big")
                                        ).hexdigest()[-40:]
B_stealth_address

'0xfed69df0a27f1dae0d7430ead82aaedfad6332bb'

Vérifions maintenant que l'adresse calculée par Alice est bien la même que celle calculée par Bob

In [12]:
B_stealth_address == A_stealth_address

True

## Calculer la clé privée de l'adresse stealth de Bob

De son côté, seul Bob peut récupérer les tokens déposés dans l'adresse stealth grâce à sa clé privée de dépense :
$ Stealth_{key}=B_{spend} + G * hash(Shared_{key})=b_{spend}*G+G * hash(Shared_{key})= G*(b_{spend}+hash(Shared_{key}))$

On voit qu'il faut multiplier G par $(b_{spend}+hash(Shared_{key}))$ pour obtenir l'adresse stealth, **c'est donc la clé privée correspondant à la clé publique stealth, connue uniquement de Bob qui connait b_{spend}**

$b_{stealth}=b_{spend}+hash(Shared)$

In [13]:
Shared = secp256k1.multiply(A_eph, b_scan)
Shared_hex = sha3.keccak_256(Shared[0].to_bytes(32, "big")+Shared[1].to_bytes(32, "big")).hexdigest()
b_stealth = b_spend + int(Shared_hex, 16)
b_stealth

39153944482575822531387237249775711740128993925789544779866399859639729033274

Il calcule ensuite la clé publique 
$B_{stealth} = b_{stealth}*G$

In [14]:
# Recipient has private key to ...
B_stealth = secp256k1.privtopub(b_stealth.to_bytes(32, "big"))
B_stealth

(67663851387124608323744162645277269585638670865381831245083336172545348387042,
 80449904826544093817252981338261706033086352950841917067356875711772573870404)

Il met finalement cette clé de stealth sous forme d'adresse.

In [15]:
B_stealthAddress_d  = "0x"+ sha3.keccak_256(B_stealth[0].to_bytes(32, "big")
                                        + B_stealth[1].to_bytes(32, "big")
                                        ).hexdigest()[-40:]
B_stealthAddress_d

'0xfed69df0a27f1dae0d7430ead82aaedfad6332bb'

In [16]:
Account.from_key((b_stealth).to_bytes(32, "big")).address

'0xfEd69Df0a27F1daE0D7430EAd82aaEdfAD6332bb'

## Optionnel: Alice peut aider Bob à scanner le registre des clés en ajoutant le premier byte de $h(Shared_{secret})$

In addition to S and stA, the sender also broadcasts the first byte of h(Q)

In [17]:
Shared_hased[0]

86

The recipient can do the the same a before without one EC Multiplication, one EC Addition and on Public Key to Address Conversion in order to check being a potential recipient.

In [18]:
Q_derived = secp256k1.multiply(A_eph, b_scan)
Q_hex_derived = sha3.keccak_256(Q_derived[0].to_bytes(32, "big")
                                +Q_derived[1].to_bytes(32, "big")
                               ).hexdigest()
Q_hashed_derived = bytearray.fromhex(Q_hex_derived)

Check view tag

In [19]:
run = Shared_hased[0] == Q_hashed_derived[0] 
run

True

In [20]:
if run:
    B_stealth = secp256k1.add(B_spend, secp256k1.privtopub(Shared_hased))
    B_stealth_address  = "0x"+ sha3.keccak_256(Stealth_key[0].to_bytes(32, "big")
                                            + Stealth_key[1].to_bytes(32, "big")
                                            ).hexdigest()[-40:]
B_stealth_address

'0xfed69df0a27f1dae0d7430ead82aaedfad6332bb'

In [21]:
B_stealth_address==A_stealth_address

True